In [2]:
# 표준 라이브러리
import os
import unicodedata
from io import BytesIO

# 서드파티 라이브러리
import numpy as np
import requests
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
import gradio as gr

In [3]:
# TrOCR 모델과 관련된 전처리기, 모델, 토크나이저를 불러옴
processor = TrOCRProcessor.from_pretrained("ddobokki/ko-trocr")  # TrOCR 전처리기
model = VisionEncoderDecoderModel.from_pretrained("ddobokki/ko-trocr")  # TrOCR 비전-텍스트 모델
tokenizer = AutoTokenizer.from_pretrained("ddobokki/ko-trocr")  # TrOCR 모델의 토크나이저

# URL에서 이미지 다운로드
url = "https://raw.githubusercontent.com/ddobokki/ocr_img_example/master/g.jpg"
response = requests.get(url)  # URL로부터 이미지 다운로드
img = Image.open(BytesIO(response.content))  # 이미지를 메모리에 로드하여 열기

# 이미지 데이터를 모델 입력값으로 전처리
pixel_values = processor(img, return_tensors="pt").pixel_values  # 이미지를 모델에 맞는 텐서로 변환

# 모델을 사용해 텍스트 생성 (OCR 수행)
generated_ids = model.generate(pixel_values, max_length=64)  # 이미지로부터 텍스트 토큰 생성
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]  # 토큰을 텍스트로 디코딩

# 텍스트 정규화 (NFC 형식)
generated_text = unicodedata.normalize("NFC", generated_text)  # 유니코드 정규화로 텍스트 통일

# 결과 출력
print(generated_text)  # 추출된 텍스트 출력

preprocessor_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/104k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/358k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/root/miniconda3/envs/my_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/427M [00:00<?, ?B/s]

VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


지방토목주사


In [4]:
# TrOCRInferencer 클래스 정의 - 모델, 전처리기, 토크나이저를 불러와 설정
class TrOCRInferencer:
    def __init__(self):
        # 초기화 시, 모델과 전처리기, 토크나이저를 미리 학습된 모델에서 불러옴
        print("[info] init TrOCR Inferencer")
        self.processor = TrOCRProcessor.from_pretrained("ddobokki/ko-trocr")
        self.model = VisionEncoderDecoderModel.from_pretrained("ddobokki/ko-trocr")
        self.tokenizer = AutoTokenizer.from_pretrained("ddobokki/ko-trocr")
        
    # 이미지에서 텍스트를 추출하는 메서드
    def inference(self, image):
        # 이미지를 텐서 형식으로 전처리
        pixel_values = self.processor(images=image, return_tensors='pt').pixel_values
        # 모델을 사용해 텍스트 토큰 생성
        generated_ids = self.model.generate(pixel_values, max_length=64)
        # 토큰을 실제 텍스트로 디코딩
        generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        # 유니코드 정규화로 텍스트 정리
        generated_text = unicodedata.normalize("NFC", generated_text)
        
        return generated_text

# TrOCRInferencer 객체 생성
inferencer = TrOCRInferencer()

[info] init TrOCR Inferencer


In [21]:
def image_to_text(image):
    # dict 형태일 경우 composite 이미지 추출
    if isinstance(image, dict):
        image = image.get('composite', None)  # 'composite' 키에 접근
        if image is None:
            return "No valid image data found"
    
    # 이미지가 NumPy 배열일 경우 처리
    image = Image.fromarray(image).convert('RGB')
    
    try:
        # 추론을 통해 텍스트 추출
        text = inferencer.inference(image)
    except Exception as e:
        return str(e)
    
    return text

In [ ]:
brush = gr.Brush(default_size = 3)

# Gradio 인터페이스 정의
with gr.Blocks() as app:
    gr.Markdown("# Handwritten Image OCR")  # 앱 제목
    # 첫 번째 탭: 이미지 업로드 방식
    with gr.Tab("Image upload"):
        image = gr.Image(label="Handritten image file")  # 이미지 업로드 컴포넌트
        output = gr.Textbox(label="Output Box")  # 결과 텍스트 박스
        convert_btn = gr.Button("Convert")  # 변환 버튼
        # 버튼 클릭 시 이미지에서 텍스트로 변환하는 함수 호출
        convert_btn.click(
            fn=image_to_text, inputs=image, outputs=output
        )
        # 예시 이미지 기능 (주석 처리됨)
        # gr.Markdown("## Image Examples")
        # gr.Examples(
        #     examples=[
        #         os.path.join(os.getcwd(), "examples/Hello.png"),
        #         os.path.join(os.getcwd(), "examples/Hello_cursive.png"),
        #         os.path.join(os.getcwd(), "examples/Red.png"),
        #         os.path.join(os.getcwd(), "examples/sentence.png"),
        #         os.path.join(os.getcwd(), "examples/i_love_you.png"),
        #         os.path.join(os.getcwd(), "examples/merrychristmas.png"),
        #         os.path.join(os.getcwd(), "examples/Rock.png"),
        #         os.path.join(os.getcwd(), "examples/Bob.png"),
        #     ],
        #     inputs=image,
        #     outputs=output,
        #     fn=image_to_text
        # )
    
    # 두 번째 탭: 스케치 패드로 손글씨 작성
    with gr.Tab("Drawing"):
        gr.Markdown("# Handwritten Image OCR")  # 탭 제목
        # 스케치패드 컴포넌트 (사용자가 손글씨를 그림)
        sketchpad = gr.Sketchpad(
            label = "Handwritten Sketchpad",  # 라벨명 수정
            width=600,  # 스케치패드 너비
            height=400,  # 스케치패드 높이
            brush=brush,
            container=True,  # 레이아웃 크기 고정
            scale=2,  # 크기 비율 설정
        )
        output = gr.Textbox(label="Output Box")  # 결과 텍스트 박스
        convert_btn = gr.Button("Convert")  # 변환 버튼
        # 버튼 클릭 시 스케치패드에서 작성한 그림을 텍스트로 변환
        convert_btn.click(
            fn=image_to_text, inputs=sketchpad, outputs=output
        )

app.launch(inline=False, share=True, debug=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://a4bab1cafe69ff7113.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
